# **Data Preparation for PBMs**

## **Import Dependencies**

In [153]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import os
from glob import glob
import json
from tqdm.auto import tqdm
import ee
import geemap
from sklearn.neighbors import BallTree

plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = 'Times New Roman'

import warnings
warnings.filterwarnings('ignore')

out_master_dir = r'datasets\master'
out_temp_dir = r'temp_data'

## **Instantiate a Map Object**

In [72]:
# ee.Authenticate()
# ee.Initialize()

In [73]:
# Import Germany Shapefile
de_roi = ee.FeatureCollection('users/geonextgis/Germany_Administrative_Level_2')
poly_style = {'fillColor': '00000000', 'color': 'black', 'width': 1}
Map = geemap.Map(basemap='Esri.WorldImagery')
Map.addLayer(de_roi.style(**poly_style), {}, 'DE ROI')
Map.centerObject(de_roi, 6)
Map

Map(center=[51.055719127031935, 10.373828310619029], controls=(WidgetControl(options=['position', 'transparent…

## **Read the Datasets**

In [74]:
# Read the DE NUTS3 Shapefile
de_nuts3_gdf = gpd.read_file("datasets\shapefiles\DE_NUTS\DE_NUTS_3.shp")
de_nuts3_gdf = de_nuts3_gdf[de_nuts3_gdf['LEVL_CODE']==3] # Filter for NUTS3
print(de_nuts3_gdf.shape)
de_nuts3_gdf.head()

(400, 6)


,NUTS_ID,LEVL_CODE,CNTR_CODE,NAME_LATN,NUTS_NAME,geometry
0,DE11B,3,DE,Main-Tauber-Kreis,Main-Tauber-Kreis,"POLYGON ((1074230.536 6408356.046, 1073820.827..."
1,DE11C,3,DE,Heidenheim,Heidenheim,"MULTIPOLYGON (((1131091.261 6235073.568, 11312..."
2,DE11D,3,DE,Ostalbkreis,Ostalbkreis,"MULTIPOLYGON (((1141777.678 6284962.486, 11412..."
3,DE121,3,DE,"Baden-Baden, Stadtkreis","Baden-Baden, Stadtkreis","MULTIPOLYGON (((910859.613 6248068.047, 913127..."
4,DE122,3,DE,"Karlsruhe, Stadtkreis","Karlsruhe, Stadtkreis","POLYGON ((938225.711 6286986.826, 940668.057 6..."


In [75]:
# Read the soil coordinates
soil_coords_df = pd.read_csv("datasets\csvs\Site_Soil_BZE_WGS84_Coords.csv")

# Read the soil attributes
soil_attributes_df = pd.read_excel(
    "datasets\csvs\SoilData.xlsx",
    sheet_name='SoilData'
)
soil_attributes_df.drop(columns=['Lon', 'Lat'], inplace=True)
soil_attributes_df.rename(columns={'Location_id': 'PointID'}, inplace=True)

print('Soil coordinates data shape:', soil_coords_df.shape)
print('Soil coordinates attributes shape:', soil_attributes_df.shape)

Soil coordinates data shape: (3104, 3)
Soil coordinates attributes shape: (3087, 31)


In [76]:
# Read the ESA WorldCover dataset
esa_lulc = ee.ImageCollection("ESA/WorldCover/v100").first()
visualization = {
  'bands': ['Map'],
}
Map.addLayer(esa_lulc, visualization, 'ESA LULC', opacity=0.6)

## **Soil Data Preparation**

In [77]:
# Convert the soil coordinates into geodataframe
soil_coords_geometry = gpd.points_from_xy(soil_coords_df['Longitude'], soil_coords_df['Latitude'], crs=4326)
soil_coords_gdf = gpd.GeoDataFrame(soil_coords_df, geometry=soil_coords_geometry).to_crs(epsg=3857)
print(soil_coords_gdf.shape)
soil_coords_gdf.head()

(3104, 4)


,PointID,Longitude,Latitude,geometry
0,2,8.411608,54.859923,POINT (936375.919 7334727.347)
1,3,8.697143,54.864382,POINT (968161.53 7335589.787)
2,4,8.765298,54.866979,POINT (975748.51 7336092.132)
3,5,8.959032,54.863920,POINT (997314.88 7335500.425)
4,6,9.078456,54.870685,POINT (1010609.099 7336809.049)


In [78]:
# Merge the NUTS3 information in the soil coordinates dataframe
soil_coords_gdf = soil_coords_gdf.sjoin(de_nuts3_gdf[['NUTS_ID', 'NUTS_NAME', 'geometry']], how='left', predicate='intersects')
print(soil_coords_gdf.shape)
soil_coords_gdf.head()

(3104, 7)


,PointID,Longitude,Latitude,geometry,index_right,NUTS_ID,NUTS_NAME
0,2,8.411608,54.859923,POINT (936375.919 7334727.347),166.0,DEF07,Nordfriesland
1,3,8.697143,54.864382,POINT (968161.53 7335589.787),166.0,DEF07,Nordfriesland
2,4,8.765298,54.866979,POINT (975748.51 7336092.132),166.0,DEF07,Nordfriesland
3,5,8.959032,54.863920,POINT (997314.88 7335500.425),166.0,DEF07,Nordfriesland
4,6,9.078456,54.870685,POINT (1010609.099 7336809.049),166.0,DEF07,Nordfriesland


In [ ]:
# Merge the soil coordinate information in the soil attributes dataframe
soil_attributes_gdf = pd.merge(
    left=soil_coords_gdf[['PointID', 'Longitude', 'Latitude', 'NUTS_ID', 'NUTS_NAME', 'geometry']], 
    right=soil_attributes_df, 
    on='PointID', 
    how='inner')

soil_attributes_gdf.columns = [col.replace('.0', '') for col in soil_attributes_gdf.columns]
soil_attributes_gdf.to_crs(crs='epsg:31467', inplace=True) # change the CRS
soil_attributes_ee = geemap.gdf_to_ee(soil_attributes_gdf, geodesic=False)

soil_style = {'color': 'green', 'pointSize': 5}
Map.addLayer(soil_attributes_ee.style(**soil_style), {}, 'Soil Points')

print(soil_attributes_gdf.shape)
soil_attributes_gdf.head()

(3087, 36)


,PointID,Longitude,Latitude,NUTS_ID,NUTS_NAME,geometry,SoilLayerDepth_10cm,SoilLayerDepth_30cm,SoilLayerDepth_50cm,SoilLayerDepth_70cm,...,BD_50cm,BD_70cm,BD_1m,BD_2m,OC_10cm,OC_30cm,OC_50cm,OC_70cm,OC_1m,OC_2m
0,2,8.411608,54.859923,DEF07,Nordfriesland,POINT (3462285.321 6081355.028),0.1,0.3,0.5,0.7,...,1.45,1.30,1.54,1.54,2.183,2.075,1.151,0.691,0.157,0.157
1,3,8.697143,54.864382,DEF07,Nordfriesland,POINT (3480623.416 6081734.949),0.1,0.3,0.5,0.7,...,1.57,1.39,1.47,1.47,2.064,1.589,0.574,0.579,0.581,0.581
2,4,8.765298,54.866979,DEF07,Nordfriesland,POINT (3485000.58 6082007.3),0.1,0.3,0.5,0.7,...,1.37,1.48,1.49,1.49,1.915,1.517,2.220,1.028,0.652,0.652
3,5,8.959032,54.863920,DEF07,Nordfriesland,POINT (3497439.177 6081642.411),0.1,0.3,0.5,0.7,...,1.45,1.48,1.60,1.60,3.375,1.701,1.233,1.089,0.674,0.674
4,6,9.078456,54.870685,DEF07,Nordfriesland,POINT (3505106.596 6082397.618),0.1,0.3,0.5,0.7,...,1.42,0.89,1.35,1.35,2.447,1.166,0.398,3.570,0.241,0.241


In [101]:
# Save the Soil data
# soil_attributes_gdf.to_csv(os.path.join(out_csv_dir, 'DE_Soil_BZE_Master.csv'), index=False)
# soil_attributes_gdf.to_file(os.path.join(out_master_dir, 'DE_Soil_BZE_Master.shp'), index=False)

## **Climate Grid Preparation**

In [97]:
# Read the DWD Climate JSON file
dwd_json_path = "datasets\shapefiles\dwd_ubn_latlon_to_rowcol.json"

with open(dwd_json_path, 'r') as file:
    dwd_json = json.load(file)

# Function to convert the json data into a table
def json_to_table(data):
    coords_info = [i[0] for i in dwd_json]
    row_col_info = [i[1] for i in dwd_json]

    coords_info = pd.DataFrame(coords_info, columns=['Latitude', 'Longitude'])
    row_col_info = pd.DataFrame(row_col_info, columns=['Row', 'Column'])

    final_df = pd.concat((coords_info, row_col_info), axis=1)

    return final_df

dwd_grid_df = json_to_table(dwd_json)
print(dwd_grid_df.shape)
dwd_grid_df.head()

(358303, 4)


,Latitude,Longitude,Row,Column
0,55.054328,8.402969,0,181
1,55.054404,8.418616,0,182
2,55.045268,8.387459,1,180
3,55.045346,8.403102,1,181
4,55.036286,8.387596,2,180


In [98]:
# Convert the data into geodataframe
dwd_grid_gdf = gpd.GeoDataFrame(
    dwd_grid_df, 
    geometry=gpd.points_from_xy(dwd_grid_df['Longitude'], dwd_grid_df['Latitude']),
    crs=4326)
print(dwd_grid_gdf.shape)
dwd_grid_gdf.head()

(358303, 5)


,Latitude,Longitude,Row,Column,geometry
0,55.054328,8.402969,0,181,POINT (8.40297 55.05433)
1,55.054404,8.418616,0,182,POINT (8.41862 55.0544)
2,55.045268,8.387459,1,180,POINT (8.38746 55.04527)
3,55.045346,8.403102,1,181,POINT (8.4031 55.04535)
4,55.036286,8.387596,2,180,POINT (8.3876 55.03629)


In [99]:
# # Save the data
# dwd_grid_df.to_csv(os.path.join(out_csv_dir, 'DE_DWD_UBN_Centroids.csv'), index=False)

# dwd_grid_gdf.to_crs(31467).to_file("datasets\shapefiles\DE_DWD_UBN_GRIDS\DE_DWD_UBN_Centroids_EPSG_31467.shp")

## **Filter Points falling on Cropland**

In [90]:
# Load the DWD grid centroids on EE Map object
dwd_centroids_ee = ee.FeatureCollection('projects/ee-geonextgis/assets/DE_DWD_UBN_Centroids_EPSG_31467') 
dwd_style = {
    'color': 'red',
    'pointSize': 1,
    'width': 1,
    'fillColor': '00000000'
}

# Create a 500m buffer for each feature and get bounds
dwd_grids_ee = dwd_centroids_ee.map(lambda f: f.buffer(500).bounds())

Map.addLayer(dwd_centroids_ee.style(**dwd_style), {}, 'DWD Centroids', False)
Map.addLayer(dwd_grids_ee.style(**dwd_style), {}, 'DWD Grids', False)

In [91]:
# # Iterate over the columns and extract the data in the temporary folder
# for column_id in tqdm(sorted(dwd_grid_gdf['Column'].unique())):

#     filered_column_cells = dwd_grids_ee.filter(ee.Filter.eq('Column', int(column_id)))

#     try:
#         out_data_path = os.path.join(out_temp_dir, f'Col_{column_id}_DWD_LULC.csv')

#         # Extract LULC info for all the DWD cells
#         dwd_lulc_zonal_stat = geemap.zonal_statistics_by_group(
#             esa_lulc, filered_column_cells, out_data_path, statistics_type='SUM'
#         )

#         print(f'Column ID: {column_id} | Data saved at {out_data_path}')

#     except:
#         print(f'Column ID: {column_id} | Error.')

In [217]:
# Merge all the data in a single file
concatenated_df = pd.DataFrame()

lulc_class_columns = ['Class_10', 'Class_20', 'Class_30', 'Class_40', 'Class_50', 'Class_60',
                      'Class_70', 'Class_80', 'Class_90', 'Class_95', 'Class_100']

temp_file_paths = glob(out_temp_dir + '\\*.csv')

for path in tqdm(temp_file_paths):
    temp_df = pd.read_csv(path)

    for col in lulc_class_columns:
        if col not in list(temp_df.columns):
            temp_df[col] = 0

        temp_df[col] = np.round((temp_df[col] / temp_df['Class_sum']) * 100, 4)

    temp_df = temp_df[['Row', 'Column', 'Longitude', 'Latitude'] + lulc_class_columns]

    concatenated_df = pd.concat((concatenated_df, temp_df), axis=0)

# Filter the grid cell where cropland ('Class_40') area is more than 20%
dwd_cropland_df = concatenated_df[concatenated_df['Class_40']>=20].iloc[:, :4]
dwd_cropland_gdf = pd.merge(left=dwd_cropland_df, right=dwd_grid_gdf[['Row', 'Column', 'geometry']], on=['Row', 'Column'], how='left')
dwd_cropland_gdf = gpd.GeoDataFrame(dwd_cropland_gdf)
print(dwd_cropland_gdf.shape)
dwd_cropland_gdf.head()

  0%|          | 0/640 [00:00<?, ?it/s]

(190658, 5)


,Row,Column,Longitude,Latitude,geometry
0,444,0,5.876024,51.024361,POINT (5.87602 51.02436)
1,477,100,7.311274,50.757258,POINT (7.31127 50.75726)
2,481,100,7.312566,50.721317,POINT (7.31257 50.72132)
3,482,100,7.312888,50.712331,POINT (7.31289 50.71233)
4,511,100,7.322172,50.451747,POINT (7.32217 50.45175)


In [218]:
# Add the NUTS information
dwd_cropland_gdf = dwd_cropland_gdf.to_crs(crs='epsg:3857')
dwd_cropland_gdf = dwd_cropland_gdf.sjoin(de_nuts3_gdf[['NUTS_ID', 'NUTS_NAME', 'geometry']], how='left', predicate='intersects')
dwd_cropland_gdf.dropna(inplace=True)
dwd_cropland_gdf.sort_values(by=['NUTS_ID'], inplace=True)
dwd_cropland_gdf.reset_index(drop=True, inplace=True)
dwd_cropland_gdf['Cell_ID'] = dwd_cropland_gdf.index
dwd_cropland_gdf = dwd_cropland_gdf[['Cell_ID', 'Row', 'Column', 'Latitude', 'Longitude', 'NUTS_ID', 'NUTS_NAME', 'geometry']]
print(dwd_cropland_gdf.shape)
dwd_cropland_gdf.head()

(190364, 8)


,Cell_ID,Row,Column,Latitude,Longitude,NUTS_ID,NUTS_NAME,geometry
0,0,703,234,48.737371,9.201742,DE111,"Stuttgart, Stadtkreis",POINT (1024333.233 6230415.593)
1,1,691,236,48.845227,9.229425,DE111,"Stuttgart, Stadtkreis",POINT (1027414.872 6248640.293)
2,2,707,233,48.701424,9.188012,DE111,"Stuttgart, Stadtkreis",POINT (1022804.862 6224350.32)
3,3,706,233,48.710416,9.188046,DE111,"Stuttgart, Stadtkreis",POINT (1022808.604 6225867.209)
4,4,705,233,48.719409,9.188080,DE111,"Stuttgart, Stadtkreis",POINT (1022812.348 6227384.366)


In [219]:
# Save the data
# dwd_cropland_gdf.to_csv(os.path.join(out_master_dir, 'DE_DWD_UBN_Crop.csv'), index=False)

## **Find k-Nearest Soil Points for Each Climate Grid Cell**

In [220]:
# Convert the soil data in the same coordinate system
soil_attributes_gdf.to_crs(crs='epsg:3857', inplace=True)
soil_attributes_gdf.crs == dwd_cropland_gdf.crs

True

In [221]:
# Extract coordinates
dwd_cropland_coords = np.array(list(zip(dwd_cropland_gdf.geometry.x, dwd_cropland_gdf.geometry.y)))
soil_coords = np.array(list(zip(soil_attributes_gdf.geometry.x, soil_attributes_gdf.geometry.y)))

# Build BallTree for fast nearest neighbor search
tree = BallTree(soil_coords, metric='euclidean')

# Define number of neighbors (k)
k = 5  # Adjust based on data density

# Query k-nearest neighbors for each climate point
distances, indices = tree.query(dwd_cropland_coords, k=k)

## **Compute Inverse Distance Weighted (IDW) Average**

In [222]:
# Define soil properties to interpolate
soil_properties = soil_attributes_gdf.columns[12:]  # Add all required properties
power = 2  # IDW power parameter

# Dictionary to store weighted values for each property
weighted_results = {prop: [] for prop in soil_properties}

# Compute IDW for each climate grid cell
for i, climate_point in tqdm(enumerate(dwd_cropland_gdf.geometry)):
    nearest_soil_points = soil_attributes_gdf.iloc[indices[i]]  # Get k nearest neighbors
    nearest_distances = distances[i]

    # Avoid division by zero
    nearest_distances[nearest_distances == 0] = 1e-6

    # Compute weights (w = 1/d^p)
    weights = 1 / (nearest_distances ** power)

    # Compute weighted average for each soil property
    for prop in soil_properties:
        weighted_avg = np.round(np.sum(weights * nearest_soil_points[prop].values) / np.sum(weights), 4)
        weighted_results[prop].append(weighted_avg)

# Assign weighted values to climate GeoDataFrame
for prop in soil_properties:
    dwd_cropland_gdf[f"{prop}"] = weighted_results[prop]

0it [00:00, ?it/s]

In [229]:
# Add soil layer depth columns (needed for PBMs)
dwd_cropland_gdf['SoilLayerDepth_10cm'] = 0.1
dwd_cropland_gdf['SoilLayerDepth_30cm'] = 0.3
dwd_cropland_gdf['SoilLayerDepth_50cm'] = 0.5
dwd_cropland_gdf['SoilLayerDepth_70cm'] = 0.7
dwd_cropland_gdf['SoilLayerDepth_1m'] = 1
dwd_cropland_gdf['SoilLayerDepth_2m'] = 2

# Reorder the columns
dwd_cropland_gdf = dwd_cropland_gdf[list(dwd_cropland_gdf.columns[:7]) + list(soil_attributes_gdf.columns[6:12]) + list(soil_properties) + ['geometry']]
print(dwd_cropland_gdf.shape)
dwd_cropland_gdf.head()

(190364, 38)


,Cell_ID,Row,Column,Latitude,Longitude,NUTS_ID,NUTS_NAME,SoilLayerDepth_10cm,SoilLayerDepth_30cm,SoilLayerDepth_50cm,...,BD_70cm,BD_1m,BD_2m,OC_10cm,OC_30cm,OC_50cm,OC_70cm,OC_1m,OC_2m,geometry
0,0,703,234,48.737371,9.201742,DE111,"Stuttgart, Stadtkreis",0.1,0.3,0.5,...,1.5407,1.5394,1.5394,1.2970,1.1749,0.3660,0.3316,0.1847,0.1847,POINT (1024333.233 6230415.593)
1,1,691,236,48.845227,9.229425,DE111,"Stuttgart, Stadtkreis",0.1,0.3,0.5,...,1.4462,1.4607,1.4607,2.2463,1.8119,0.6600,0.5122,0.2873,0.2873,POINT (1027414.872 6248640.293)
2,2,707,233,48.701424,9.188012,DE111,"Stuttgart, Stadtkreis",0.1,0.3,0.5,...,1.5569,1.4867,1.4867,1.3170,1.1219,0.3438,0.3209,0.1713,0.1713,POINT (1022804.862 6224350.32)
3,3,706,233,48.710416,9.188046,DE111,"Stuttgart, Stadtkreis",0.1,0.3,0.5,...,1.5501,1.5007,1.5007,1.2979,1.1741,0.3449,0.3250,0.1761,0.1761,POINT (1022808.604 6225867.209)
4,4,705,233,48.719409,9.188080,DE111,"Stuttgart, Stadtkreis",0.1,0.3,0.5,...,1.5484,1.5136,1.5136,1.3000,1.1741,0.3491,0.3254,0.1780,0.1780,POINT (1022812.348 6227384.366)


In [232]:
# Save the data
# dwd_cropland_gdf.drop(columns='geometry').to_csv(os.path.join(out_master_dir, 'DE_DWD_UBN_Crop_Soil.csv'), index=False)